In [16]:
%pip install tensorflow


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [17]:
import pandas as pd
import os
import numpy as np

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [6]:
jokes_df = pd.read_csv('jokes.csv')
jokes_df.head()

,ID,Joke
0,1,"[me narrating a documentary about narrators] ""..."
1,2,Telling my daughter garlic is good for you. Go...
2,3,I've been going through a really rough period ...
3,4,"If I could have dinner with anyone, dead or al..."
4,5,Two guys walk into a bar. The third guy ducks.


In [18]:
print("Number of records: ", jokes_df.shape[0])
print("Number of fields: ", jokes_df.shape[1])

Number of records:  231657
Number of fields:  2


In [8]:
jokes_df['Joke']


0         [me narrating a documentary about narrators] "...
1         Telling my daughter garlic is good for you. Go...
2         I've been going through a really rough period ...
3         If I could have dinner with anyone, dead or al...
4            Two guys walk into a bar. The third guy ducks.
                                ...                        
231652                  The Spicy Sausage by Delia Katessen
231653    TIL That I Shouldn't have gone to law school, ...
231654    What did the RAM stick say to the politician? ...
231655    what do you call a play about victorian era me...
231656    Calculus should be taught in every high school...
Name: Joke, Length: 231657, dtype: object

In [9]:

jokes_df['Joke'] = jokes_df['Joke'].apply(lambda x: x.replace(u'\xa0',u' '))
jokes_df['Joke'] = jokes_df['Joke'].apply(lambda x: x.replace('\u200a',' '))

In [10]:
tokenizer = Tokenizer(oov_token='<oov>') # For those words which are not found in word_index
tokenizer.fit_on_texts(jokes_df['Joke'])
total_words = len(tokenizer.word_index) + 1

print("Total number of words: ", total_words)
print("Word: ID")
print("------------")
print("<oov>: ", tokenizer.word_index['<oov>'])
print("Strong: ", tokenizer.word_index['strong'])
print("And: ", tokenizer.word_index['and'])
print("Consumption: ", tokenizer.word_index['consumption'])

Total number of words:  70650
Word: ID
------------
<oov>:  1
Strong:  1479
And:  7
Consumption:  9242


In [11]:
input_sequences = []
for line in jokes_df['Joke']:
    token_list = tokenizer.texts_to_sequences([line])[0]
    #print(token_list)
    
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# print(input_sequences)
print("Total input sequences: ", len(input_sequences))

Total input sequences:  3850485


In [12]:
# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
input_sequences[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,   15, 9606,    2], dtype=int32)

In [13]:
# create features and label
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]


In [15]:
# Assuming total_words, max_sequence_len, xs, and ys are already defined
model = Sequential()
model.add(Embedding(total_words, 2))  # Removed input_length argument
model.add(Bidirectional(LSTM(64)))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(learning_rate=0.01)  # Updated learning rate setting
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(xs, labels, epochs=1, batch_size=128)  

30082/30082 ━━━━━━━━━━━━━━━━━━━━ 5128s 170ms/step - accuracy: 0.0898 - loss: 6.6380


In [38]:
from keras.preprocessing.sequence import pad_sequences
import numpy as np

# Example seed text
seed_text = "why did"
# Number of words you want to generate - adjust based on how long you want the joke to be
next_words = 25

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')
    predicted_probs = model.predict(token_list, verbose=0)
    predicted_index = np.argmax(predicted_probs, axis=-1)[0]
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted_index:
            output_word = word
            break
    seed_text += " " + output_word
    # Optional: Stop if a sentence-ending punctuation is generated
    if output_word in ['.']:
        break

print(seed_text)


why did the chicken cross the road to the other because he was a little medium in the same side of the same time of the world


In [43]:
from keras.preprocessing.sequence import pad_sequences
import numpy as np

# Example: Starting with two words
seed_text = "Why do"
next_words = 15 # You may want to adjust this based on your observation

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')
    predicted_probs = model.predict(token_list, verbose=0)
    predicted_index = np.argmax(predicted_probs, axis=-1)[0]
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted_index:
            output_word = word
            break
    seed_text += " " + output_word
    # Optionally end the loop if the model outputs sentence-ending punctuation
    if output_word in ['.', '!', '?']:
        break

print(seed_text)


Why do you get a good because they have a little joke of the other and the


In [46]:
from keras.preprocessing.sequence import pad_sequences
import numpy as np

seed_text = "donald trump"  # Your initial seed text
end_tokens = ['.', '!', '?']  # Tokens that might indicate the end of a joke

for _ in range(10):  # A high upper limit to ensure we don't loop indefinitely
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')
    predicted_probs = model.predict(token_list, verbose=0)
    predicted_index = np.argmax(predicted_probs, axis=-1)[0]
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted_index:
            output_word = word
            break
    seed_text += " " + output_word
    # Break the loop if the output word suggests the end of a joke
    if output_word in end_tokens:
        break

print(seed_text)


donald trump is a lot of a bar and a man and
